In [10]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns


In [11]:
df = pd.read_csv("https://raw.githubusercontent.com/atharvabhide/Quora-Question-Pair-Similarity/refs/heads/main/notebooks/train.csv")
df.sample(5)


,id,qid1,qid2,question1,question2,is_duplicate
277346,277346,396442,103188,What do people think of Timesvr.com?,What do people think of jobdreaming.com?,0
199826,199826,79391,96722,Which is a good solar panel installation provi...,Which is a good solar panel installation provi...,1
16290,16290,31065,31066,"What is the status of ""Good Samaritan laws"" in...",What benefits will the enactment of Good Samar...,0
268953,268953,263783,386609,Why does induction motor draw heavy current at...,"I am running a Orphanage past 15 years, Now I ...",0
253051,253051,367517,280391,Where is eduardo saverin now?,Who is Eduardo Saverin?,0


In [12]:
df.shape


(404290, 6)

In [13]:
new_df = df.sample(30000)


In [14]:
new_df.isnull().sum()


id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [15]:
new_df.duplicated().sum()


0

In [16]:
ques_df = new_df[['question1', 'question2']]
ques_df.head()


,question1,question2
278813,Should Donald Trump publish his tax returns?,Should Donald Trump publish his tax-returns?
309545,How can I find the right girl to get married?,How do I find the right girl for me?
32432,Is it possible to gave a good product but fail...,Is it possible to have a good product but fail...
175999,Why is MiG-21 being phased out? Can its airfra...,Why can't the Mig 21s being phased out of IAF ...
137069,What is the best thing that you have bought?,What is the best thing that you have ever bought?


In [17]:
from sklearn.feature_extraction.text import CountVectorizer

# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(), 2)


In [18]:
temp_df1 = pd.DataFrame(q1_arr, index=ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index=ques_df.index)

temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.sample(5)


,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
274277,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
379654,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69386,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
233659,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44827,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
temp_df.shape


(30000, 6000)

In [21]:
temp_df['is_duplicate'] = new_df['is_duplicate']
temp_df.sample(5)


,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
9125,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
78853,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
261821,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49315,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
345523,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(temp_df.iloc[:, 0:-1].values, temp_df.iloc[:, -1].values, test_size=0.2, random_state=42)


In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)


0.744

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier()

xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test, y_pred)


0.7281666666666666

: 